In [ ]:
import torch
import torch.nn as nn

class FIGARCH_CNN(nn.Module):
    def __init__(self, truncation_size, init_beta=0.5, init_phi=0.5, init_d=0.5, init_omega = 0.3):
        super(FIGARCH_CNN, self).__init__()
        self.truncation_size = truncation_size
        # Параметры FI-GARCH (обучаемые)
        self.beta = nn.Parameter(torch.tensor(init_beta))
        self.phi = nn.Parameter(torch.tensor(init_phi))
        self.d = nn.Parameter(torch.tensor(init_d))
        self.omega = nn.Parameter(torch.tensor(init_omega))
        # Инициализация весов свёртки (lambda_k)
        self.register_buffer('lambdas', torch.zeros(truncation_size))
        self.update_lambdas()  # Первоначальный расчёт lambda_k

    def update_lambdas(self):
        """Обновляет веса lambda_k на основе текущих beta, phi, d."""
        deltas = torch.ones(self.truncation_size)
        lambdas = torch.zeros(self.truncation_size)
        
        # Рекуррентный расчёт lambda_k
        lambdas[0] = 1.0  # lambda_0
        if self.truncation_size > 1:
            lambdas[1] = self.phi - self.beta + self.d
            deltas[1] = (1 - self.d) / 1
        
        for k in range(2, self.truncation_size):
            deltas[k] = deltas[k-1] * (k - 1 - self.d) / k
            lambdas[k] = self.beta * lambdas[k-1] + ((k - 1 - self.d)/k - self.phi) * deltas[k-1]
        
        self.lambdas = lambdas.detach()  # Отключаем градиенты для lambda_k

    def forward(self, x):
        """x: тензор размером (batch_size, seq_len), где seq_len >= truncation_size."""
        # Применяем свёртку с весами lambda_k
        x = x.unfold(dimension=1, size=self.truncation_size, step=1)
        sigma_sq = torch.sum(x * self.lambdas.flip(0), dim=2)  # Скалярное произведение
        return sigma_sq

# Пример использования:
truncation_size = 10
model = FIGARCH_CNN(truncation_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Пример данных: (batch_size, seq_len)
epsilon_sq = torch.rand(32, 100)  # 32 батча, 100 временных точек

for epoch in range(100):
    model.update_lambdas()  # Обновляем lambda_k перед каждым forward
    sigma_sq_pred = model(epsilon_sq)
    loss = torch.mean((sigma_sq_pred - true_sigma_sq)**2)  # Пример MSE
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()